In [19]:
#Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
#Acceder al archivo csv
dataframe = pd.read_csv('DSCancerDeMamaSinDatosPerdidos.csv', sep=';')

In [23]:
#Definimos los valores para entrenar al algoritmo
y=dataframe['SEVERITY']
x=dataframe.drop('SEVERITY',axis=1)

In [25]:
#Entrenar el modelo
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=30)

from sklearn.neighbors import KNeighborsClassifier
knn_mm = KNeighborsClassifier(n_neighbors=3)

knn_mm.fit(x_train,y_train)

predicciones=knn_mm.predict(x_test)

In [48]:
#Librerías para la conección entre el formulario HTML y el archivo Jupiter
from flask import Flask, render_template, request

In [53]:
#Nombre del módulo actual
app = Flask(__name__)

In [55]:
#Ruta que permite la comunicación con el formulario y métodos que le son permitidos
@app.route("/", methods=["GET", "POST"])
#Definimos la función de respuesta
def formulario():
    resultado = None
    try:
        if request.method == "POST":
            #Datos recibidos del formulario
            BIRADS = int(request.form["BIRADS"])
            AGE = int(request.form["AGE"])
            SHAPE = int(request.form["SHAPE"])
            MARGIN = int(request.form["MARGIN"])
            DENSITY = int(request.form["DENSITY"])

            #Respuesta del algoritmo
            prediccion = knn_mm.predict(pd.DataFrame([[BIRADS,AGE,SHAPE,MARGIN,DENSITY]],columns=x.columns))
            prob_prediccion = knn_mm.predict_proba(pd.DataFrame([[BIRADS,AGE,SHAPE,MARGIN,DENSITY]],columns=x.columns))

            #Generando respuesta según los resultados de la predicción
            match int(prediccion[0]):
                case 0:
                    probabilidad = round(float(prob_prediccion[0][0])*100,2)
                    resultado = f"El resultado es benigno con una probabilidad de {probabilidad}%."
                case 1:
                    probabilidad = round(float(prob_prediccion[0][0])*100,2)
                    resultado = f"El resultado es maligno con una probabilidad de {probabilidad}%."
            print("Respuesta exitosa")
    
    except ValueError:
        resultado = "Hubo un error X_X"
        print("Error detectado")

    #Se busca el archivo formulario.html en la carpeta templates y se envía de respuesta
    return render_template("formulario.html", resultado=resultado)

In [57]:
#Ejecución del archivo
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/May/2025 23:09:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2025 23:09:55] "POST / HTTP/1.1" 200 -


Respuesta exitosa
